In [2]:
from sympy import symbols, init_printing
import sympy
import sympy.physics.mechanics as me
init_printing(use_latex='mathjax')
import seaborn as sns
# sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
import matplotlib.pyplot as plt
from scipy.integrate import ode, odeint
import numpy as np
%matplotlib inline
import InputShaping as shaping

In [3]:
# Create the variables
x, y, beta = me.dynamicsymbols('x, y, beta')

# Create the velocities
x_dot, y_dot, beta_dot = me.dynamicsymbols('x, y, beta', 1)

# Create the constants
m, k, L, g, H, c, D,t, Izz, k_beta, c_beta = sympy.symbols('m k L g H c D t Izz k_beta c_beta')
L_1_init, L_2_init, Le1, Le2 = sympy.symbols('L_1_init L_2_init Le1 Le2')
# m = mass
# k = spring k
# L = spring equilibrium length
# g = gravity
# c = spring c
# c_beta = rotational c
# k_beta = rotational k
# D = rod length
# Izz = moment of Inertia about the end of a rod

In [48]:
# Create the world frame
N = me.ReferenceFrame('N')

# Create the rod frame
B = N.orientnew('B', 'axis', [beta, N.z])

# Set the rotation of the rod frame
B.set_ang_vel(N, beta_dot * N.z)

# Create the Origin
O1 = me.Point('O_1')

# Set origin velocity to zero
O1.set_vel(N, 0 * N.x)

# Create the second attachment point
O2 = O1.locatenew('O_2', H * N.x)
O2.set_vel(N, 0 * N.x)

# Locate the point in the N frame
# P = me.Point('pen')
P = O1.locatenew('P', x * N.x + y * N.y)

# P.set_pos(O1, x * N.x + y * N.y)

# Set the point's velocity
P.set_vel(N, x_dot * N.x + y_dot * N.y)

# Create the rod center of mass
G = P.locatenew('G', D/2 * B.y)

# Set the velocity of G
G.v2pt_theory(P, N, B)

# Create the rod
I_rod = me.inertia(B, 0, 0, Izz)
rod = me.RigidBody('rod', G, B, m, (I_rod, G))

# Create the distance from the point to each attachment point
L1 = O1.pos_from(P).magnitude
L2 = O2.pos_from(P).magnitude
L1_vector = O1.pos_from(P).normalize
L2_vector = O2.pos_from(P).normalize

F1 = (-Le1 + L1()) * k * L1_vector()
F2 = (-Le2 + L2()) * k * L2_vector()

# Create the height from the center of gravity to the datum
h = G.pos_from(O1) & N.y

In [55]:
def True_length(x,y):
    H = 20.0
    g = 9.81
    k = 10.0
    m = 1.0
#     y = y - 1.81446563 - 0.3333
    
    Fx = (-k*(-Le1 + np.sqrt(x**2 + y**2))*x/np.sqrt(x**2 + y**2) + 
         k*(H - x)*(-Le2 + np.sqrt((H - x)**2 + y**2))/np.sqrt((H - x)**2 + y**2))
    
    Fy = (-k*(-Le1 + np.sqrt(x**2 + y**2))*y/np.sqrt(x**2 + y**2) + 
          -k*(-Le2 + np.sqrt((H - x)**2 + y**2))*y/np.sqrt((H - x)**2 + y**2)) + m*g
    
    solution = sympy.solvers.solve([Fx,Fy])
    L1 = solution[Le1] 
    L2 = solution[Le2] 
    return L1,L2

In [58]:
L1, L2 = True_length(9,17)
print("Length 1 is: {}\n".format(L1))
print("Length 2 is: {}\n".format(L2))

Length 1 is: 18.6248869163493

Length 2 is: 19.7226519534319



In [51]:
print(L1().subs({x:10,y:10,H:20}))
print(L2().subs({x:10,y:10,H:20}))

10*sqrt(2)
10*sqrt(2)


In [52]:
print(F1)
print(F2)

- k*(-Le1 + sqrt(x**2 + y**2))*x/sqrt(x**2 + y**2)*N.x - k*(-Le1 + sqrt(x**2 + y**2))*y/sqrt(x**2 + y**2)*N.y
k*(H - x)*(-Le2 + sqrt((H - x)**2 + y**2))/sqrt((H - x)**2 + y**2)*N.x - k*(-Le2 + sqrt((H - x)**2 + y**2))*y/sqrt((H - x)**2 + y**2)*N.y


In [9]:
# sympy.physics.vector.dot()
Fx1 = F1.subs({x:15,y:10,H:20, k:10}) & N.x

In [10]:
Fx2 = F2.subs({x:15,y:10,H:20, k:10}) & N.x

In [11]:
Fy1 = F1.subs({x:15,y:10,H:20, k:10}) & N.y

In [12]:
Fy2 = F2.subs({x:15,y:10,H:20, k:10}) & N.y

In [13]:
Fx = Fx1 + Fx2
Fys = Fy1 + Fy2 + m*g
Fy = Fys.subs({m:10.0, g:9.81})

In [14]:
Fx

  30⋅√13⋅(Le₁ - 5⋅√13)                    
- ──────────────────── + 2⋅√5⋅(Le₂ - 5⋅√5)
           13                             

In [15]:
Fy

  20⋅√13⋅(Le₁ - 5⋅√13)                           
- ──────────────────── - 4⋅√5⋅(Le₂ - 5⋅√5) + 98.1
           13                                    

In [16]:
sympy.solvers.solve([Fx,Fy])

{Le₁: 22.4490636288577, Le₂: 19.4062749597263}

In [101]:
ass[Le1]

14.1883804072206